In [1]:
import gzip
from pathlib import Path
from shutil import copyfileobj

import healpy
import hipscat_import.pipeline as runner
import numpy as np
import pandas as pd
from astropy.coordinates import SkyCoord
from dustmaps.sfd import SFDQuery
from hipscat_import.catalog.arguments import ImportArguments
from tqdm import tqdm

from utils import get_hemisphere

In [2]:
ORDER = 14
BATCH_SIZE = 1 << 20

NSIDE = healpy.order2nside(ORDER)
NPIX = healpy.nside2npix(NSIDE)

NBATCHES = int(np.ceil(NPIX / BATCH_SIZE))
NBATCHES_LEN = len(str(NBATCHES - 1))

In [3]:
DATA_DIR = Path('./data')
INPUT_DIR = DATA_DIR / 'input'
PARQUET_DIR = DATA_DIR / 'tmp-parquet'
OUTPUT_DIR = DATA_DIR / 'output'

Decompress the input files for `dustmaps` and create `SFDQuery` instance

In [4]:
for gz_file in INPUT_DIR.glob('*.gz'):
    decompressed = INPUT_DIR / gz_file.stem
    if decompressed.exists():
        continue
    with gzip.open(gz_file, 'rb') as f_in:
        with open(decompressed, 'wb') as f_out:
            copyfileobj(f_in, f_out)

In [5]:
sfd = SFDQuery(INPUT_DIR)

Generate pixel list, coordinates and E(B-V)

In [ ]:
def gen_pixel_list():
    for i in range(0, NPIX, BATCH_SIZE):
        yield i, min(i + BATCH_SIZE, NPIX)
        
def ebv(pixels):
    ra, dec = healpy.pix2ang(NSIDE, pixels, nest='ring', lonlat=True)
    coord = SkyCoord(ra=ra, dec=dec, unit='deg')
    return sfd(coord)
    
def write_parquet(filename, pixels, values):
    df = pd.DataFrame({'pixel': pixels, 'ebv': values})
    df.to_parquet(filename, index=False)
    
for i, (start, end) in tqdm(enumerate(gen_pixel_list()), total=NBATCHES):
    pixels = np.arange(start, end)
    values = ebv(pixels)
    filename = PARQUET_DIR / f'{i:0{NBATCHES_LEN}}.parquet'
    write_parquet(filename, pixels, values)

/Users/hombit/.virtualenvs/hipscat-import-sfd/lib/python3.11/site-packages/distributed/client.py:3163: UserWarning: Sending large graph of size 128.01 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
